# Use PMML to predict iris species with `ibm-watsonx-ai`

This notebook contains steps from storing sample PMML model to starting scoring new data. 

Some familiarity with python is helpful. This notebook uses Python 3.11.

You will use a **Iris** data set, which details measurements of iris perianth. Use the details of this data set to predict iris species.

## Learning goals

The learning goals of this notebook are:

-  Working with the WML instance
-  Online deployment of PMML model
-  Scoring of deployed model


## Contents

This notebook contains the following parts:

1. [Setup](#setup)
2. [Model upload](#upload) 
3. [Web service creation](#deploy)
4. [Scoring](#score)
5. [Clean up](#cleanup)
6. [Summary and next steps](#summary)

<a id="setup"></a>
## 1. Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Contact with your Cloud Pack for Data administrator and ask him for your account credentials

### Install and import the `ibm-watsonx-ai` and dependecies
**Note:** `ibm-watsonx-ai` documentation can be found <a href="https://ibm.github.io/watsonx-ai-python-sdk/index.html" target="_blank" rel="noopener no referrer">here</a>.

In [ ]:
!pip install wget| tail -n 1
!pip install -U ibm-watsonx-ai | tail -n 1

### Connection to WML

Authenticate the `watsonx.ai` service on IBM Cloud Pack for Data. You need to provide platform `url`, your `username` and `api_key`.

In [ ]:
username = 'PASTE YOUR USERNAME HERE'
api_key = 'PASTE YOUR API_KEY HERE'
url = 'PASTE THE PLATFORM URL HERE'

In [ ]:
from ibm_watsonx_ai import Credentials

credentials = Credentials(
    username=username,
    api_key=api_key,
    url=url,
    instance_id="openshift",
    version="5.1"
)

Alternatively you can use `username` and `password` to authenticate WML services.

```python
credentials = Credentials(
    username=***,
    password=***,
    url=***,
    instance_id="openshift",
    version="5.1"
)

```

In [3]:
from ibm_watsonx_ai import APIClient

client = APIClient(credentials)

### Working with spaces

First of all, you need to create a space that will be used for your work. If you do not have space already created, you can use `{PLATFORM_URL}/ml-runtime/spaces?context=icp4data` to create one.

- Click New Deployment Space
- Create an empty space
- Go to space `Settings` tab
- Copy `space_id` and paste it below

**Tip**: You can also use SDK to prepare the space for your work. More information can be found [here](https://github.com/IBM/watson-machine-learning-samples/blob/master/cpd5.1/notebooks/python_sdk/instance-management/Space%20management.ipynb).

**Action**: Assign space ID below

In [4]:
space_id = 'PASTE YOUR SPACE ID HERE'

You can use `list` method to print all existing spaces.

In [ ]:
client.spaces.list(limit=10)

To be able to interact with all resources available in `watsonx.ai`, you need to set **space** which you will be using.

In [5]:
client.set.default_space(space_id)

'SUCCESS'

<a id="upload"></a>
## 2. Upload model

In this section you will learn how to upload the model to the Cloud.

**Action**: Download sample PMML model from git project using wget.

In [ ]:
import os
from wget import download

sample_dir = 'pmml_sample_model'
if not os.path.isdir(sample_dir):
    os.mkdir(sample_dir)
    
filename=os.path.join(sample_dir, 'iris_chaid.xml')
if not os.path.isfile(filename):
    filename = download('https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cpd5.1/models/pmml/iris-species/model/iris_chaid.xml', out=sample_dir)

Store downloaded file in `watsonx.ai` repository.

In [7]:
sw_spec_uid = client.software_specifications.get_uid_by_name("pmml-3.0_4.3")

meta_props = {
    client.repository.ModelMetaNames.NAME: "pmmlmodel",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: sw_spec_uid,
    client.repository.ModelMetaNames.TYPE: 'pmml_4.2.1'}

In [8]:
published_model = client.repository.store_model(model=filename, meta_props=meta_props)

**Note:** You can see that model is successfully stored in `watsonx` Service.

In [ ]:
client.repository.list_models()

<a id="deploy"></a>
## 3. Create online deployment

You can use commands bellow to create online deployment for stored model (web service).

In [10]:
model_id = client.repository.get_model_id(published_model)
deployment = client.deployments.create(
    artifact_uid=model_id,
    meta_props={
        client.deployments.ConfigurationMetaNames.NAME: "Test deployment",
        client.deployments.ConfigurationMetaNames.ONLINE:{}}
)



######################################################################################

Synchronous deployment creation for id: '29f8c066-4fb5-4df2-9ed9-7ae9155274fe' started

######################################################################################


initializing
Note: online_url is deprecated and will be removed in a future release. Use serving_urls instead.
......
ready


-----------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_id='98d0c7c8-4ad8-4e0b-a514-6b7961118bbd'
-----------------------------------------------------------------------------------------------




<a id="score"></a>
## 4. Scoring

You can send new scoring records to web-service deployment using `score` method.

In [12]:
import json

deployment_id = client.deployments.get_id(deployment)

scoring_data = {
    client.deployments.ScoringMetaNames.INPUT_DATA: [
        {
            'fields': ['Sepal.Length', 'Sepal.Width', 'Petal.Length', 'Petal.Width'],
            'values': [[5.1, 3.5, 1.4, 0.2]]
        }]
}

predictions = client.deployments.score(deployment_id, scoring_data)
print(json.dumps(predictions, indent=2))

{
  "predictions": [
    {
      "fields": [
        "$R-Species",
        "$RC-Species",
        "$RP-Species",
        "$RP-setosa",
        "$RP-versicolor",
        "$RP-virginica",
        "$RI-Species"
      ],
      "values": [
        [
          "setosa",
          1.0,
          1.0,
          1.0,
          0.0,
          0.0,
          "1"
        ]
      ]
    }
  ]
}


As we can see this is Iris Setosa flower.

<a id="cleanup"></a>
## 5. Clean up 

If you want to clean up all created assets:
- experiments
- trainings
- pipelines
- model definitions
- models
- functions
- deployments

please follow up this sample [notebook](https://github.com/IBM/watson-machine-learning-samples/blob/master/cpd5.1/notebooks/python_sdk/instance-management/Machine%20Learning%20artifacts%20management.ipynb).

<a id="summary"></a>
## 6. Summary and next steps     

 You successfully completed this notebook! You learned how to use `watsonx.ai` for PMML model deployment and scoring. 
 
Check out our _<a href="https://ibm.github.io/watsonx-ai-python-sdk/samples.html" target="_blank" rel="noopener no referrer">Online Documentation</a>_ for more samples, tutorials, documentation, how-tos, and blog posts. 

### Authors

**Lukasz Cmielowski**, PhD, is a Software Architect and Data Scientist at IBM.

Copyright © 2020-2025 IBM. This notebook and its source code are released under the terms of the MIT License.